In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.time import Time

In [2]:
from schmutz import trappist1, transit_duration

In [3]:
from rms import Star, Planet, Spot, STSP

In [4]:
spot_contrast = 0.25
rotation_period = 3.3 * u.day
planet = trappist1('b')
planet.lam = 0
planet.per_rot = rotation_period.value
planet.b = np.cos(np.radians(planet.inc)) * planet.a
planet.duration = transit_duration(planet)

star = Star(u=[0.161, 0.208], rotation_period=rotation_period, 
            inc_stellar=90, spot_contrast=spot_contrast, 
            rho_s=51.1, planet=planet)

In [ ]:
random_latitude = lambda: 180 * np.random.rand() - 90
random_longitude = lambda: 360 * np.random.rand()
# radius = np.arctan(np.radians(2)) # Solar-like spot case
radius = np.arctan(np.radians(7)) # Giant spot case
halfptp = 0.002433

frequency of spot occultations vs spot size 
amplitude of spot occultations vs spot size 

In [ ]:
n_radii = 10
n_contrasts = 10
radii = np.arctan(np.radians(np.linspace(1.5, 10, n_radii)))
contrasts = np.linspace(0.1, 0.9, n_contrasts)
n_iterations = 100
max_residuals = []
median_residuals = []
times = Time(np.arange(planet.t0 - transit_duration(planet)/2, 
                       planet.t0 + transit_duration(planet)/2, 1/60/24), format='jd')

# Spotless light curve
tiny_polar_spot = [Spot(89*u.deg, 0*u.deg, 0.00001)]
with STSP(times, star, tiny_polar_spot, quiet=True) as stsp: 
    lc_nospots = stsp.generate_lightcurve(normalize_oot=True)

n_significant = np.zeros((len(radii), len(contrasts)))
n_occultations = np.zeros((len(radii), len(contrasts)))
n_models = np.zeros((len(radii), len(contrasts)))

n_iterations_completed = 0

for j, spot_contrast in enumerate(contrasts):
    star = Star(u=[0.161, 0.208], rotation_period=rotation_period, 
            inc_stellar=90, spot_contrast=spot_contrast, 
            rho_s=51.1, planet=planet)

    for i, radius in enumerate(radii):
        n_max_sn = 0
        for _ in range(n_iterations):
            spotted_area = 0
            spots = []
            while spotted_area < 0.08:
                new_spot = Spot(random_latitude()*u.deg, random_longitude()*u.deg, radius)
                spots.append(new_spot)
                spotted_area = np.sum([s.radius**2 for s in spots])/4

            with STSP(times, star, spots, quiet=True) as stsp: 
                lc = stsp.generate_lightcurve(normalize_oot=True)

            if len(lc.fluxes) == len(lc_nospots.fluxes):
                residuals = lc.fluxes - lc_nospots.fluxes

                if np.nanmax(residuals) > 0:
                    n_occultations[i, j] += 1

                    if np.nanmax(residuals) > 2*halfptp: 
                        n_significant[i, j] += 1
#     max_residuals.append(np.max(peak_residuals))
#     median_residuals.append(np.median(peak_residuals))
                n_models[i, j] += 1 

In [ ]:
n_significant, n_occultations, n_models

In [ ]:
# fig, ax = plt.subplots(2, 1, figsize=(4, 6), sharex=True)
# ax[0].plot(radii, max_residuals)
# ax[0].set(title='Max peak transit residual', ylabel='Relative Flux')
# ax[1].plot(radii, median_residuals)
# ax[1].set(title='Median peak transit residual', xlabel='$R_p/R_\star$', ylabel='Relative Flux')

In [ ]:
np.savetxt('outputs/radii.txt', radii)
np.savetxt('outputs/contrasts.txt', contrasts)
np.savetxt('outputs/n_significant.txt', n_significant)
np.savetxt('outputs/n_occultations.txt', n_occultations)
np.savetxt('outputs/n_models.txt', n_models)